In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import cv2

In [2]:
labels = pd.read_csv("frames\\labels.csv")

In [3]:
labels["width"] = abs(labels.loc[:, "xmin"] - labels.loc[:, "xmax"])

In [4]:
labels["height"] = abs(labels.loc[:, "ymin"] - labels.loc[:, "ymax"])

In [5]:
labels

,xmin,ymin,xmax,ymax,Frame,Label,Preview URL,width,height
0,785,533,905,644,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,120,111
1,89,551,291,680,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,202,129
2,268,546,383,650,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,115,104
3,455,522,548,615,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,93,93
4,548,522,625,605,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,77,83
5,1726,484,1919,646,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,193,162
6,758,557,807,617,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,49,60
7,633,561,680,597,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,47,36
8,682,557,718,593,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...,36,36
9,710,540,836,665,1479498372942264998.jpg,Car,http://crowdai.com/images/c3AY-ptjXfT_oC9F/vis...,126,125


In [6]:
from glob import glob

In [7]:
frames = glob("frames\\*.jpg")

In [30]:
for idx, file in enumerate(frames):
    if (idx%10 == 0):
        frame = file.split("\\")[-1]
        image = mpimg.imread(file)
        frame_cars = labels[(labels["Frame"] == frame) & (labels["Label"] == "Car")]
        for i, row in frame_cars.iterrows():
            if (min(row["width"], row["height"]) >= 64):
                side = max(row["width"], row["height"])
                xstart = min(row["xmin"], row["xmax"])
                ystart = min(row["ymin"], row["ymax"])
                patch = image[ystart:min(ystart+side, image.shape[0]), xstart:min(xstart+side, image.shape[1]), :]
                resized = cv2.resize(patch, (64, 64))
                mpimg.imsave("frames/car/" + str(idx) + str(i) + ".png", resized, format='png')

In [35]:
def coord_is_car(x, y, frame_df):
    for idx, row in frame_df.iterrows():
        xlow = min(row["xmin"], row["xmax"])
        xhigh = max(row["xmin"], row["xmax"])
        ylow = min(row["ymin"], row["ymax"])
        yhigh = max(row["ymin"], row["ymax"])
        if ((xhigh > x &  xlow < x) & (yhigh > y) & (ylow < y)):
            return True
        return False

In [37]:
for idx, file in enumerate(frames):
    if (idx%2 == 0):
        frame = file.split("\\")[-1]
        image = mpimg.imread(file)
        frame_cars = labels[(labels["Frame"] == frame) & (labels["Label"] == "Car")]
        x = np.random.randint(low = 0, high = 1920-64, size = 10)
        y = np.random.randint(low = 700, high=1100-64, size = 10)
        for xcoord, ycoord in zip(x,y):
            if (not coord_is_car(xcoord + 32, ycoord + 32, frame_cars)):
                mpimg.imsave("frames/not_car/" + str(idx) + str(i) + ".png", image[ycoord: ycoord + 64, xcoord:xcoord+64, :], format='png')